In [ ]:
import numpy as np
import cv2
import os

from keras.models import Sequential
from keras.layers import Conv1D, Conv2D, Dense, Flatten, Dropout, MaxPooling2D
from keras.utils import np_utils
from keras.optimizers import RMSprop
from sklearn.externals import joblib
from sklearn import cluster

import tensorflow as tf
import keras.backend.tensorflow_backend as KTF

config = tf.ConfigProto()  
config.gpu_options.allow_growth=True   #不全部占满显存, 按需分配
sess = tf.Session(config=config)

# 設定 Keras 使用的 TensorFlow Session
KTF.set_session(sess)
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

animeface_cascade        = cv2.CascadeClassifier('lbpcascade_animeface.xml')
frontalface_cascade      = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
head_and_shouler_cascade = cv2.CascadeClassifier('HS.xml')


INPUT_SIZE   = 64
INPUT_SIZE_2 = int(INPUT_SIZE / 2)
INPUT_SIZE_8 = int(INPUT_SIZE / 8)
OUTPUT_SIZE  = 1

LABEL_NUM = 7
CROPS = ['NO', 'UL', 'UR', 'DL', 'DR', 'MID', 'EDGE']
PLAN  = '2DCNN'

def My_Crop (IMG, C):
    
    if   (C == CROPS[0]): # NO
        return IMG
    elif (C == CROPS[1]): # UL
        return cv2.resize(IMG[0:INPUT_SIZE_2, 0:INPUT_SIZE_2], (INPUT_SIZE, INPUT_SIZE))
    elif (C == CROPS[2]): # UR
        return cv2.resize(IMG[0:INPUT_SIZE_2, INPUT_SIZE_2:INPUT_SIZE_2*2], (INPUT_SIZE, INPUT_SIZE))
    elif (C == CROPS[3]): # DL
        return cv2.resize(IMG[INPUT_SIZE_2:INPUT_SIZE_2*2, 0:INPUT_SIZE_2], (INPUT_SIZE, INPUT_SIZE))
    elif (C == CROPS[4]): # DR
        return cv2.resize(IMG[INPUT_SIZE_2:INPUT_SIZE_2*2, INPUT_SIZE_2:INPUT_SIZE_2*2], (INPUT_SIZE, INPUT_SIZE))
    elif (C == CROPS[5]): # MID
        return cv2.resize(IMG[INPUT_SIZE_8:INPUT_SIZE_2+INPUT_SIZE_8*3, INPUT_SIZE_8:INPUT_SIZE_2+INPUT_SIZE_8*3], (INPUT_SIZE, INPUT_SIZE))
    elif (C == CROPS[6]): # EDGE
        return np.repeat(cv2.Canny(cv2.cvtColor(IMG, cv2.COLOR_BGR2GRAY), 200, 255), 3)
    else:
        return np.zeros(INPUT_SIZE * INPUT_SIZE * 3)

Using TensorFlow backend.


# 動漫人臉偵測 by lbpcascade_animeface.xml

In [106]:
def Get_Image_Animeface_Detection (IP):
    
    img  = cv2.imread(IP)
    if img is None:
        return [img, []]
    
    print(img.shape)    
    temp = 256 / img.shape[0]
    gray = cv2.cvtColor(cv2.resize(img, (int(img.shape[1] * temp), int(img.shape[0] * temp))), cv2.COLOR_BGR2GRAY)
    
    #img = cv2.resize(img, (int(img.shape[1] * temp), int(img.shape[0] * temp)))
    #gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    """
    list_faces = []
    
    faces1 = animeface_cascade.detectMultiScale(gray, 1.1, 1)
    if(len(faces1) > 0):
        list_faces.append(faces1.tolist())
    
    faces2 = frontalface_cascade.detectMultiScale(gray, 1.1, 1)
    if(len(faces2) > 0):
        list_faces.append(faces2.tolist())
        
    faces3 = head_and_shouler_cascade.detectMultiScale(gray, 1.1, 1)
    if(len(faces3) > 0):
        list_faces.append(faces3.tolist())
    
    faces = np.array(eval('[%s]'%repr(list_faces).replace('[', '').replace(']', ''))).reshape(-1, 4)
    """
    
    faces = animeface_cascade.detectMultiScale(gray, 1.1, 1)
    if(len(faces) > 0):
        faces = (faces / temp).astype(np.int32)
    
    return [img, faces]

In [108]:
dIMAGE_PATH = 'data_extra/Google/dealing/0.jpg'

img, faces = Get_Image_Animeface_Detection (dIMAGE_PATH)

for (x,y,w,h) in faces:
    cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),2)
    
cv2.imshow('img',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

error: OpenCV(3.4.4) C:\projects\opencv-python\opencv\modules\highgui\src\window.cpp:356: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'cv::imshow'


# 動畫人臉擷取 by lbpcascade_animeface.xml

In [4]:
D_FRAME  = 30  # N楨取一次
MIN_EDGE = 200 # 

def Crop_Video_Animeface_Detection (IP, OP, NAME):

    # 開啟影片檔案
    cap = cv2.VideoCapture(IP)

    if not os.path.exists(OP):
        os.makedirs(OP)

    # 以迴圈從影片檔案讀取影格，並顯示出來
    image_i = 0
    frame_i = 0
    while(cap.isOpened()):
    
        ret, img = cap.read()

        frame_i = frame_i + 1
        if(frame_i % D_FRAME != 0):
            continue

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = animeface_cascade.detectMultiScale(gray, 1.2, 3)

        # 取出所有偵測的結果
        for (x, y, w, h) in faces:

            if(w < MIN_EDGE):
                continue

            face = cv2.resize(img[y:y+h, x:x+w], (256, 256))
            cv2.imwrite(OP + '/' + str(image_i) + '-' + NAME + '.jpg', face)
            image_i = image_i + 1

    cap.release()

In [31]:
VIDEO_NAME = '1_EroManga_Sensei'
VIDEO_TYPE = '.mp4'

Crop_Video_Animeface_Detection (VIDEO_NAME+VIDEO_TYPE, VIDEO_NAME, VIDEO_NAME)

error: OpenCV(3.4.2) c:\miniconda3\conda-bld\opencv-suite_1534379934306\work\modules\imgproc\src\color.hpp:253: error: (-215:Assertion failed) VScn::contains(scn) && VDcn::contains(dcn) && VDepth::contains(depth) in function 'cv::CvtHelper<struct cv::Set<3,4,-1>,struct cv::Set<1,-1,-1>,struct cv::Set<0,2,5>,2>::CvtHelper'


# 篩選訓練資料 by k-means

In [5]:
def getFirst(elem):
    return elem[0]

def Choose_Data_by_Kmeans (IP, OP, N):
    
    for dCHARACTER_DIR_NAME in os.listdir(IP):
    
        label, name = dCHARACTER_DIR_NAME.split('_', 1)
        list_imgs   = []
    
        IP_d = IP + '/' + dCHARACTER_DIR_NAME
        for dEPISODE in os.listdir(IP_d):
        
            IP_dd = IP_d + '/' + dEPISODE
            for dIMAGE in os.listdir(IP_dd):
            
                img = cv2.resize(cv2.imread(IP_dd + '/' + dIMAGE), (INPUT_SIZE, INPUT_SIZE)).ravel()
                list_imgs.append(img)
    
        imgs = np.array(list_imgs).astype(np.float32) / 255
        print(dCHARACTER_DIR_NAME, imgs.shape)
        #print('imgs :\n', imgs)
    
        if (imgs.shape[0] >= N):
            
            if not os.path.exists(OP):
                os.makedirs(OP)
            
            KMEANS_MODEL = cluster.KMeans(n_clusters = N, random_state = 42)
            imgs_kmeans = KMEANS_MODEL.fit(imgs)
            #print('imgs_kmeans :\n', imgs_kmeans.labels_)

            labs_imgs = list(zip(imgs_kmeans.labels_, imgs))
            labs_imgs.sort(key=getFirst)
            #print('labs_imgs :\n', labs_imgs)

            choose_i = 0
            for ele in labs_imgs:
                
                if (ele[0] == choose_i):

                    img_ = (np.array(ele[1]) * 255).reshape(INPUT_SIZE, INPUT_SIZE, 3).astype(np.int32)

                    OP_d = OP + '/' + dCHARACTER_DIR_NAME
                    if not os.path.exists(OP_d):
                        os.makedirs(OP_d)
                    cv2.imwrite(OP_d + '/' + str(choose_i) + '.jpg', img_)

                    choose_i += 1

In [33]:
dIMAGE_DIR_PATH = 'data'
dTRAIN_DIR_PATH = 'data_train'
IMAGE_OUTNUM = 60

Choose_Data_by_Kmeans ('./'+dIMAGE_DIR_PATH, './'+dTRAIN_DIR_PATH, IMAGE_OUTNUM)

0_Izumi Sagiri (370, 12288)
1_Izumi Masamune (848, 12288)
2_Yamada Erufu (246, 12288)
3_Jinno Megumi (172, 12288)
4_Muramasa Senju (137, 12288)
5_Takasago Tomoe (60, 12288)
6_Kagurazaka Ayame (88, 12288)


# 建立模型 by CNN

In [6]:
def Build_Model (IP, C, P, TRAIN_N, TEST_N):

    list_labels        = []
    list_train_numbers = []
    list_test_numbers  = []

    list_train_images  = []
    list_test_images   = []

    for dCHARACTER_DIR_NAME in os.listdir(IP):
    
        label, name = dCHARACTER_DIR_NAME.split('_', 1)

        list_labels.append(int(label))
        list_train_numbers.append(TRAIN_N)
        list_test_numbers.append(TEST_N)

        i = 0
        IP_d = IP + '/' + dCHARACTER_DIR_NAME
        for dIMAGE in os.listdir(IP_d):
            
            img_rgb = cv2.imread(IP_d + '/' + dIMAGE)
            img = My_Crop(img_rgb, C)
        
            if(i < TRAIN_NUM):
                list_train_images.append(img.tolist())
            else:
                list_test_images.append(img.tolist())
            i = i + 1
        
    labels        = np.array(list_labels       ).reshape(-1)
    train_numbers = np.array(list_train_numbers).reshape(-1)
    test_numbers  = np.array(list_test_numbers ).reshape(-1)
    
    if   (P == '1DCNN'):
        train_images = np.array(list_train_images).reshape(-1, INPUT_SIZE*INPUT_SIZE*3).astype(np.float32) / 255
        test_images  = np.array(list_test_images ).reshape(-1, INPUT_SIZE*INPUT_SIZE*3).astype(np.float32) / 255

        train_images = np.expand_dims(train_images, axis=2)
        test_images  = np.expand_dims(test_images,  axis=2)

    elif (P == '2DCNN'):
        train_images = np.array(list_train_images).reshape(-1, INPUT_SIZE, INPUT_SIZE*3).astype(np.float32) / 255
        test_images  = np.array(list_test_images ).reshape(-1, INPUT_SIZE, INPUT_SIZE*3).astype(np.float32) / 255

        train_images = np.expand_dims(train_images, axis=3)
        test_images  = np.expand_dims(test_images,  axis=3)

    train_labels = np_utils.to_categorical(labels.repeat(train_numbers), len(labels)).astype(np.uint32)
    test_labels  = np_utils.to_categorical(labels.repeat(test_numbers),  len(labels)).astype(np.uint32)


    print("train_images.shape : ", train_images.shape)
    print("test_images.shape  : ", test_images.shape)
    print("train_labels.shape : ", train_labels.shape)


    # build model
    model = Sequential()

    if  (P == '1DCNN'):
        model.add(Conv1D(filters=16, kernel_size=8, activation='relu', input_shape=(INPUT_SIZE*INPUT_SIZE*3, 1)))
        model.add(Flatten())
        model.add(Dense(OUTPUT_SIZE))
        model.compile(loss='mse', optimizer='adam')
        train_history = model.fit(train_images, train_labels, epochs=40, batch_size=10, shuffle=False, validation_data=(test_images, test_labels), verbose=2)

    elif (P == '2DCNN'):

        model.add(Conv2D(filters=16, kernel_size=(3,3), padding='same', activation='relu', input_shape=(INPUT_SIZE, INPUT_SIZE*3, 1)))
        model.add(MaxPooling2D(pool_size=(2,2)))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(len(labels), activation='softmax'))
        rmsprop = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
        model.compile(optimizer=rmsprop, loss='categorical_crossentropy', metrics=['accuracy'])
        #model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        train_history = model.fit(train_images, train_labels, epochs=100, batch_size=30, shuffle=False, 
                                  validation_data=(test_images, test_labels), verbose=2)

    model.save(C + '_' + P + '.h5')


In [20]:
dTRAIN_DIR_PATH = 'data_train'
CROP = 'EDGE'
TRAIN_NUM = 50
TEST_NUM  = 10

Build_Model (dTRAIN_DIR_PATH, CROP, PLAN, TRAIN_NUM, TEST_NUM)

train_images.shape :  (350, 64, 192, 1)
test_images.shape  :  (70, 64, 192, 1)
train_labels.shape :  (350, 7)
Train on 350 samples, validate on 70 samples
Epoch 1/100
 - 1s - loss: 6.8959 - acc: 0.1543 - val_loss: 4.1235 - val_acc: 0.1429
Epoch 2/100
 - 0s - loss: 1.9821 - acc: 0.4429 - val_loss: 2.9416 - val_acc: 0.1429
Epoch 3/100
 - 0s - loss: 1.3406 - acc: 0.5457 - val_loss: 2.6070 - val_acc: 0.1571
Epoch 4/100
 - 0s - loss: 0.9637 - acc: 0.6057 - val_loss: 2.3611 - val_acc: 0.2429
Epoch 5/100
 - 0s - loss: 0.6602 - acc: 0.7143 - val_loss: 2.1594 - val_acc: 0.2714
Epoch 6/100
 - 0s - loss: 0.3986 - acc: 0.8800 - val_loss: 1.8915 - val_acc: 0.3571
Epoch 7/100
 - 0s - loss: 0.2050 - acc: 0.9771 - val_loss: 1.6750 - val_acc: 0.4143
Epoch 8/100
 - 0s - loss: 0.0994 - acc: 1.0000 - val_loss: 1.4705 - val_acc: 0.4714
Epoch 9/100
 - 0s - loss: 0.0551 - acc: 1.0000 - val_loss: 1.4168 - val_acc: 0.5143
Epoch 10/100
 - 0s - loss: 0.0493 - acc: 1.0000 - val_loss: 2.0835 - val_acc: 0.4000
Epoc

Epoch 93/100
 - 0s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 0.7849 - val_acc: 0.8571
Epoch 94/100
 - 0s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 0.7849 - val_acc: 0.8571
Epoch 95/100
 - 0s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 0.7849 - val_acc: 0.8571
Epoch 96/100
 - 0s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 0.7941 - val_acc: 0.8571
Epoch 97/100
 - 0s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 0.7941 - val_acc: 0.8571
Epoch 98/100
 - 0s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 0.7876 - val_acc: 0.8429
Epoch 99/100
 - 0s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 0.7876 - val_acc: 0.8429
Epoch 100/100
 - 0s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 0.7876 - val_acc: 0.8429


In [ ]:
# 2DLDA
"""
    train_images = np.array(list_train_images).reshape(-1,INPUT_SIZE).astype(np.float32) / 255
    train_labels = np.array(list_train_labels).reshape(-1).astype(np.int32)
    test_images  = np.array(list_test_images ).reshape(-1,INPUT_SIZE).astype(np.float32) / 255
    test_labels  = np.array(list_test_labels ).reshape(-1).astype(np.int32)

    # First LDA
    first_lda = LDA(n_components=LDA_NUM, shrinkage = 0.6, solver='eigen')
    lda_1_train_images = first_lda.fit_transform(train_images, train_labels).astype(np.float32)
    lda_1_test_images  = first_lda.transform(test_images).astype(np.float32)
    joblib.dump(first_lda, 'FirstLDA_model.pkl')

    DX = lda_1_train_images.shape[1]

    train_images = lda_1_train_images.reshape(-1, INPUT_SIZE, DX).swapaxes(1,2).reshape(-1, INPUT_SIZE)
    test_images  = lda_1_test_images.reshape(-1, INPUT_SIZE, DX).swapaxes(1,2).reshape(-1, INPUT_SIZE)
    train_labels = labels.repeat(train_number * DX).astype(np.int32)

    # Second LDA
    second_lda = LDA(n_components=LDA_NUM, shrinkage = 0.6, solver='eigen')
    lda_2_train_images = second_lda.fit_transform(train_images, train_labels).astype(np.float32)
    lda_2_test_images  = second_lda.transform(test_images).astype(np.float32)
    joblib.dump(second_lda, 'SecondLDA_model.pkl')

    DY = lda_2_train_images.shape[1]

    train_images = lda_2_train_images.reshape(-1, DX, DY).swapaxes(1,2)
    test_images  = lda_2_test_images.reshape(-1, DX, DY).swapaxes(1,2)

    model = Sequential()
    model.add(Conv2D(filters=8, kernel_size=2, activation='relu', input_shape=(DY, DY, 1)))
    model.add(Flatten())
    model.add(Dense(OUTPUT_SIZE))
    model.compile(loss='mse', optimizer='adam')
    train_history = model.fit(train_images, train_labels, epochs=50, batch_size=5, shuffle=False, validation_data=(test_images, test_labels), verbose=2)
    model.save('2DLDA_2DCNN_model.h5')
"""

In [7]:
from keras.models import load_model

LABEL_NUM = 7
CROP_LIST  = ['NO', 'MID', 'EDGE']
THRESHOLD_P = 0.5

MARK = 'L' + str(LABEL_NUM) + '-C' + str(len(CROP_LIST)) + '-P' + str(int(THRESHOLD_P*10)) + '-PREDICT_'

for CROP in CROP_LIST:
    
    if   (CROP == CROPS[0]):
        NN0 = load_model(CROPS[0] + '_' + PLAN + '.h5')
    elif (CROP == CROPS[1]):
        NN1 = load_model(CROPS[1] + '_' + PLAN + '.h5')
    elif (CROP == CROPS[2]):
        NN2 = load_model(CROPS[2] + '_' + PLAN + '.h5')
    elif (CROP == CROPS[3]):
        NN3 = load_model(CROPS[3] + '_' + PLAN + '.h5')
    elif (CROP == CROPS[4]):
        NN4 = load_model(CROPS[4] + '_' + PLAN + '.h5')
    elif (CROP == CROPS[5]):
        NN5 = load_model(CROPS[5] + '_' + PLAN + '.h5')
    elif (CROP == CROPS[6]):
        NN6 = load_model(CROPS[6] + '_' + PLAN + '.h5')

# 動畫人臉分類 (圖片)

In [92]:
# IMG = INPUT_SIZE * INPUT_SIZE
def Get_Image_Animeface_Recognition (IMG, CS, P):
    
    vote = np.zeros(LABEL_NUM)
    
    for CROP in CS:
        
        IMG_ = My_Crop(IMG, CROP)
        
        if   (P == '1DCNN'):
            test_image = np.array(IMG_).reshape(1,INPUT_SIZE * INPUT_SIZE * 3).astype(np.float32) / 255
            test_image = np.expand_dims(test_image, axis=2)
        elif (P == '2DCNN'):
            test_image = np.array(IMG_).reshape(1,INPUT_SIZE, INPUT_SIZE * 3).astype(np.float32) / 255
            test_image = np.expand_dims(test_image, axis=3)
        
        if   (CROP == CROPS[0]):
            list_predicts =  NN0.predict(test_image)[0]
        elif (CROP == CROPS[1]):
            list_predicts =  NN1.predict(test_image)[0]
        elif (CROP == CROPS[2]):
            list_predicts =  NN2.predict(test_image)[0]
        elif (CROP == CROPS[3]):
            list_predicts =  NN3.predict(test_image)[0]
        elif (CROP == CROPS[4]):
            list_predicts =  NN4.predict(test_image)[0]
        elif (CROP == CROPS[5]):
            list_predicts =  NN5.predict(test_image)[0]
        elif (CROP == CROPS[6]):
            list_predicts =  NN6.predict(test_image)[0]
            
        print(CROP, ' : ', list_predicts)
        if (np.max(list_predicts) > THRESHOLD_P):
            vote[np.argmax(list_predicts)] += 1
    
    print('VOTE : ', vote)
    
    return 'unknow' if np.max(vote) < (len(CS) / 2) else str(np.argmax(vote))

In [98]:
dIMAGE_PATH = 'data_extra/Google/Test_2018-12-29_05\'11\'14/000006.jpg'

img, faces = Get_Image_Animeface_Detection (dIMAGE_PATH)

image_i = 0
for (x,y,w,h) in faces:
    
    print(str(image_i))
    image_i += 1
    
    print('ans : ', Get_Image_Animeface_Recognition (cv2.resize(img[y:y+h, x:x+w], (INPUT_SIZE,INPUT_SIZE)), CROP_LIST, PLAN))

(1080, 1917, 3)
0
NO  :  [1.0796193e-08 2.1115327e-03 1.4084467e-15 6.6012106e-11 1.3894268e-05
 9.9787378e-01 7.9066604e-07]
MID  :  [3.8391155e-07 1.8087841e-05 7.2378669e-15 7.7358280e-11 4.5090776e-05
 9.9993348e-01 2.9346850e-06]
EDGE  :  [1.0630532e-06 7.4605229e-03 5.9737306e-07 4.1258179e-08 3.9434817e-06
 9.9253368e-01 6.3332116e-08]
VOTE :  [0. 0. 0. 0. 0. 3. 0.]
ans :  5
1
NO  :  [1.6996060e-01 3.1929318e-04 1.9928905e-04 8.8981995e-09 6.3826113e-09
 5.5476325e-03 8.2397318e-01]
MID  :  [9.9580979e-01 1.8334651e-06 8.3923817e-07 1.2898727e-09 1.1028827e-09
 3.8019966e-04 3.8074087e-03]
EDGE  :  [2.0174691e-02 1.3460173e-08 9.7766703e-01 3.2378316e-09 9.5147130e-14
 4.2998985e-07 2.1577952e-03]
VOTE :  [1. 0. 1. 0. 0. 0. 1.]
ans :  unknow


# 動畫人臉分類 (圖片集)

In [9]:
MIN_EDGE = 100

def Images_Animeface_Recognition (IP, OP, CS, P):
    
    image_i = 0
    for dIMAGE in os.listdir(IP):
        
        img, faces = Get_Image_Animeface_Detection(IP + '/' + dIMAGE)
        
        for (x, y, w, h) in faces:
            
            if(w < MIN_EDGE):
                continue
                
            print(str(image_i))
            
            img_face = img[y:y+h, x:x+w]
            img_rgb  = cv2.resize(img_face, (INPUT_SIZE, INPUT_SIZE))
            
            ans_str = Get_Image_Animeface_Recognition (img_rgb, CS, P)
    
            if not os.path.exists(OP):
                os.makedirs(OP)

            if not os.path.exists(OP + '/' + ans_str):
                os.makedirs(OP + '/' + ans_str)

            cv2.imwrite(OP + '/' + ans_str + '/' + str(image_i) + '.jpg', img_face)
            image_i += 1

In [33]:
dIMAGE_DIR_NAME  = 'data_test'

for dCHARACTER_DIR_NAME in os.listdir(dIMAGE_DIR_NAME):
        
    Images_Animeface_Recognition (dIMAGE_DIR_NAME + '/' + dCHARACTER_DIR_NAME, MARK + dIMAGE_DIR_NAME + '/' + dCHARACTER_DIR_NAME, CROP_LIST, PLAN)

0
NO  :  [9.9999976e-01 1.0465647e-11 1.9927359e-07 2.0434383e-11 3.2025988e-16
 1.3154469e-13 4.5757451e-10]
MID  :  [1.0000000e+00 1.5399896e-12 1.2092533e-10 3.4496135e-15 4.0601532e-18
 2.6973083e-11 1.2621298e-09]
EDGE  :  [9.2451286e-04 3.6324253e-11 3.1075332e-01 6.8832213e-01 7.0789261e-21
 3.8267813e-12 8.3060900e-09]
VOTE :  [2. 0. 0. 1. 0. 0. 0.]
1
NO  :  [1.0000000e+00 1.8270300e-15 2.6747960e-08 7.5886469e-15 4.1355445e-18
 2.8767743e-18 3.4853660e-08]
MID  :  [1.0000000e+00 5.8070150e-13 1.0305135e-09 1.8011051e-14 2.0586160e-16
 6.5312285e-14 9.7033173e-09]
EDGE  :  [1.8887115e-06 6.3384824e-14 6.6773194e-01 2.2826507e-08 1.0508679e-20
 1.9114706e-10 3.3226618e-01]
VOTE :  [2. 0. 1. 0. 0. 0. 0.]
2
NO  :  [1.0000000e+00 2.2343912e-18 2.8817355e-09 1.2376836e-15 9.9871761e-19
 2.5270074e-16 8.8880341e-11]
MID  :  [1.0000000e+00 2.0338885e-15 8.6385554e-10 1.1758665e-17 3.7164514e-18
 1.3083290e-12 7.9130942e-09]
EDGE  :  [2.3799338e-03 7.5648732e-10 9.8913723e-01 8.4794164

MID  :  [1.0000000e+00 1.4692602e-13 5.4966822e-09 2.3535744e-13 6.8271639e-16
 4.8466378e-15 2.6924807e-08]
EDGE  :  [8.3995771e-01 1.4208680e-10 3.0224936e-04 1.5971619e-01 9.5762995e-11
 2.1478542e-08 2.3796325e-05]
VOTE :  [3. 0. 0. 0. 0. 0. 0.]
24
NO  :  [1.0000000e+00 1.0244399e-17 4.5147899e-11 5.6023945e-15 1.9038331e-19
 1.4221434e-15 3.6106989e-11]
MID  :  [1.0000000e+00 4.3843493e-14 7.8402181e-12 6.9414482e-17 1.0501093e-20
 1.6403300e-12 3.5099257e-10]
EDGE  :  [4.9814726e-03 2.5052020e-09 9.3642098e-01 4.8477161e-03 8.2417554e-12
 5.3700104e-02 4.9685434e-05]
VOTE :  [2. 0. 1. 0. 0. 0. 0.]
25
NO  :  [1.0000000e+00 1.1974502e-12 1.6737695e-10 3.6039593e-13 9.8615354e-17
 7.9885798e-14 3.4585293e-10]
MID  :  [1.0000000e+00 1.3224930e-14 1.1753813e-11 5.0647913e-18 7.1747986e-20
 4.1779714e-13 1.2192869e-09]
EDGE  :  [9.8949921e-01 3.2709502e-11 9.7033000e-03 3.3113531e-06 2.1578144e-14
 4.4026810e-06 7.8983430e-04]
VOTE :  [3. 0. 0. 0. 0. 0. 0.]
26
NO  :  [9.9999952e-01 4.1

47
NO  :  [9.9988854e-01 5.3916598e-12 3.7649643e-05 1.0619162e-11 3.3213506e-12
 2.8502818e-11 7.3759147e-05]
MID  :  [9.9992800e-01 1.5356127e-10 1.0800709e-06 3.0132476e-11 1.5830172e-10
 3.3141631e-10 7.0922339e-05]
EDGE  :  [9.9961483e-01 1.8499768e-12 2.5889708e-06 4.5904762e-06 1.2220000e-23
 3.3527507e-16 3.7791755e-04]
VOTE :  [3. 0. 0. 0. 0. 0. 0.]
48
NO  :  [1.0000000e+00 4.6623945e-20 4.3783536e-12 1.8179286e-16 2.6139625e-18
 5.8111521e-19 5.0803979e-14]
MID  :  [1.0000000e+00 1.0167047e-16 2.0901716e-10 1.6760317e-17 1.8673106e-17
 2.8105148e-14 1.9911935e-11]
EDGE  :  [9.8831159e-01 1.0057455e-08 1.1677261e-02 3.2052347e-07 7.8542789e-06
 2.9712762e-06 1.9651234e-09]
VOTE :  [3. 0. 0. 0. 0. 0. 0.]
49
NO  :  [1.0000000e+00 3.3619206e-16 2.5649402e-10 4.6121080e-18 8.9715525e-20
 8.3849580e-20 4.2781795e-11]
MID  :  [1.00000000e+00 1.33299515e-14 1.53706547e-10 1.88161977e-16
 9.07167787e-17 3.96651037e-17 2.05653453e-10]
EDGE  :  [9.9999785e-01 2.8483571e-09 7.5063832e-07

70
NO  :  [9.9999905e-01 1.7107584e-15 7.4826130e-07 7.4053312e-15 5.2702195e-19
 5.7697775e-18 2.3007213e-07]
MID  :  [1.0000000e+00 6.0871642e-16 1.4183303e-10 5.1160335e-19 3.1462116e-21
 1.7335030e-15 1.4119521e-10]
EDGE  :  [8.9938825e-01 6.8758048e-09 9.9763766e-02 1.2610369e-07 1.1262274e-05
 1.3104422e-05 8.2357164e-04]
VOTE :  [3. 0. 0. 0. 0. 0. 0.]
71
NO  :  [1.0000000e+00 1.8410215e-16 1.7002071e-08 1.5593717e-14 5.5947946e-14
 4.8415906e-16 6.4093891e-10]
MID  :  [1.0000000e+00 9.9061435e-16 6.6060908e-09 5.2033092e-17 3.2604093e-18
 2.9487797e-13 9.8064623e-10]
EDGE  :  [5.7286805e-01 4.2799220e-10 9.9941448e-04 4.2598295e-01 2.6400934e-11
 1.4713274e-04 2.4684900e-06]
VOTE :  [3. 0. 0. 0. 0. 0. 0.]
72
NO  :  [1.00000000e+00 1.54609212e-14 8.68913830e-10 8.69213759e-14
 1.09747156e-17 1.16177782e-16 4.66387906e-09]
MID  :  [1.0000000e+00 7.5289959e-13 1.7250628e-11 3.6092230e-16 1.2229964e-18
 4.0843506e-13 1.0726871e-09]
EDGE  :  [9.7514157e-06 2.7540987e-06 9.9732876e-01

NO  :  [1.0000000e+00 3.2974503e-15 2.7107117e-08 7.1458396e-14 3.2094065e-19
 2.9984094e-18 7.0014955e-10]
MID  :  [1.0000000e+00 1.0079941e-13 2.1253357e-10 1.3919732e-16 1.1217660e-19
 9.7087306e-15 5.1909043e-11]
EDGE  :  [2.73017585e-02 2.82983201e-05 7.55336225e-01 1.17515884e-01
 8.09252185e-07 9.94720683e-02 3.44997534e-04]
VOTE :  [2. 0. 1. 0. 0. 0. 0.]
95
NO  :  [9.9995589e-01 5.8435440e-10 2.1623889e-07 4.1216797e-05 2.9907385e-14
 1.4728857e-12 2.6027690e-06]
MID  :  [1.0000000e+00 3.1455266e-14 2.5104612e-13 4.4224455e-18 5.7283000e-22
 1.2182308e-14 2.5897198e-11]
EDGE  :  [6.8962657e-01 6.4781266e-03 2.5637200e-04 2.0849569e-01 7.8038983e-02
 1.2302705e-06 1.7102947e-02]
VOTE :  [3. 0. 0. 0. 0. 0. 0.]
96
NO  :  [9.9999988e-01 3.3662450e-16 1.2406204e-07 9.9311151e-16 8.4713221e-18
 2.1746136e-16 4.6582287e-11]
MID  :  [9.9999988e-01 2.8961451e-16 6.3499165e-08 9.0245579e-18 4.5102741e-17
 2.6769975e-14 2.2140423e-09]
EDGE  :  [3.6246177e-02 4.9559185e-06 9.6214527e-01 1.

2
NO  :  [7.4826882e-09 1.4736824e-08 4.9229267e-15 1.9653687e-13 9.9883777e-01
 2.0314411e-04 9.5915166e-04]
MID  :  [5.2905467e-04 1.2070655e-04 4.7187454e-10 7.2389916e-09 9.8663944e-01
 2.9088449e-05 1.2681761e-02]
EDGE  :  [4.8783506e-09 1.6600547e-04 1.1714234e-08 7.3714688e-08 9.9168772e-01
 6.9150650e-03 1.2311557e-03]
VOTE :  [0. 0. 0. 0. 3. 0. 0.]
3
NO  :  [4.3759080e-19 9.3753822e-21 9.9999988e-01 1.0634515e-07 4.8279964e-24
 7.0141656e-25 3.5167185e-11]
MID  :  [4.0435887e-23 6.3740959e-21 1.0000000e+00 1.9765497e-11 1.8163337e-24
 7.2611603e-31 2.0076094e-16]
EDGE  :  [9.5857940e-06 1.4485456e-18 9.9999034e-01 8.0341001e-10 8.2408872e-26
 1.5680812e-17 7.7725311e-08]
VOTE :  [0. 0. 3. 0. 0. 0. 0.]
4
NO  :  [1.0000000e+00 7.9565247e-14 1.7215582e-08 5.8565913e-13 9.8631157e-17
 1.5777155e-16 2.3520835e-10]
MID  :  [1.00000000e+00 3.07748347e-13 1.25978552e-08 2.91934056e-15
 2.40037517e-16 7.32216683e-14 1.09602785e-10]
EDGE  :  [9.9993408e-01 1.8172552e-11 1.3100408e-08 5.

# 動畫人臉分類 (影片)

In [10]:
D_FRAME  = 60  # N楨取一次
MIN_EDGE = 200

def Video_Animeface_Recognition (IP, OP, CS, P):
    
    # 開啟影片檔案
    cap = cv2.VideoCapture(IP)
    
    # 以迴圈從影片檔案讀取影格，並顯示出來
    image_i = 0
    frame_i = 0
    while(cap.isOpened()):
        
        ret, img = cap.read()
        
        frame_i = frame_i + 1
        if(frame_i % D_FRAME != 0):
            continue
        
        gray  = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = animeface_cascade.detectMultiScale(gray, 1.2, 3)
        
        for (x, y, w, h) in faces:
            
            if(w < MIN_EDGE):
                continue
            
            print(str(image_i))
            
            img_face = img[y:y+h, x:x+w]
            img_rgb  = cv2.resize(img_face, (INPUT_SIZE, INPUT_SIZE))
            
            ans_str = Get_Image_Animeface_Recognition (img_rgb, CS, P)
    
            if not os.path.exists(OP):
                os.makedirs(OP)

            if not os.path.exists(OP + '/' + ans_str):
                os.makedirs(OP + '/' + ans_str)

            cv2.imwrite(OP + '/' + ans_str + '/' + str(image_i) + '.jpg', img_face)
            image_i += 1

    cap.release()

In [37]:
VIDEO_NAME  = '12_EroManga_Sensei'
VIDEO_NAME  = '1_EroManga_Sensei_Coser'
VIDEO_TYPE  = '.mp4'

Video_Animeface_Recognition (VIDEO_NAME + VIDEO_TYPE, MARK + VIDEO_NAME, CROP_LIST, PLAN)

0
NO  :  [9.9999273e-01 1.0037631e-11 2.5502898e-06 4.0241952e-11 1.1489674e-10
 2.5676993e-12 4.7260251e-06]
MID  :  [9.9996734e-01 2.6631974e-09 1.2600947e-05 1.3576807e-12 1.3393471e-12
 3.6996192e-10 2.0012279e-05]
EDGE  :  [8.4145314e-01 4.9524631e-02 6.7722490e-03 6.4957654e-04 1.0853044e-03
 7.6761156e-02 2.3753760e-02]
VOTE :  [3. 0. 0. 0. 0. 0. 0.]
1
NO  :  [9.9999988e-01 1.0902750e-14 6.3342398e-08 6.7816323e-17 2.6809088e-16
 5.5666236e-12 9.0151371e-09]
MID  :  [9.9999964e-01 8.2303364e-13 2.2941670e-07 2.0900472e-14 6.9778947e-14
 1.5484319e-10 7.4630726e-08]
EDGE  :  [9.8139470e-05 3.3592804e-07 1.0098464e-04 3.5890579e-07 9.9896514e-01
 8.3506422e-04 5.0514569e-08]
VOTE :  [2. 0. 0. 0. 1. 0. 0.]
2
NO  :  [9.9999988e-01 1.0338231e-12 1.7701110e-07 1.5360095e-11 2.3746057e-12
 1.4610205e-11 9.4417301e-09]
MID  :  [9.9999511e-01 1.2425396e-09 4.6460132e-06 1.1734402e-11 6.9504675e-12
 3.7671197e-10 2.4289056e-07]
EDGE  :  [9.1525924e-01 6.2522255e-02 4.4122980e-06 1.6556615

error: OpenCV(3.4.4) C:\projects\opencv-python\opencv\modules\imgproc\src\color.cpp:181: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


# 網站爬圖 by BeautifulSoup

In [12]:
import requests
from bs4 import BeautifulSoup
import os

def getHtmlurl(url): #獲取網址
    try:
        r=requests.get(url)
        r.raise_for_status()
        r.encoding=r.apparent_encoding
        return r.text
    except:
        return ""

def getpic(html): #獲取圖片地址並下載
    soup =BeautifulSoup(html,'html.parser')
    all_img=soup.find_all('img')
    for img in all_img:
        src=img['src']
        img_url=src
        print (img_url)
        root='data_extra/NET2/'
        path = root + img_url.split('/')[-1]
        print (path)
        try: #創建或判斷路徑圖片是否存在並下載
            if not os.path.exists(root):
                os.mkdir(root)
            if not os.path.exists(path):
                r = requests.get(img_url)
                with open(path, 'wb') as f:
                    f.write(r.content)
                    f.close()
                    print("文件保存成功")
            else:
                print("文件已存在")
        except:
            print("爬取失敗")



def main():
    url='http://asamai.weebly.com/'
    html=(getHtmlurl(url))

    print(getpic(html))
    
main()

http://www.f-counter.net/j/22/1420026157/
data_extra/NET2/
文件已存在
/uploads/2/3/2/0/23202642/1470581952208.jpg?193
data_extra/NET2/1470581952208.jpg?193
爬取失敗
//cdn2.editmysite.com/images/site/footer/footer-toast-published-image-1.png
data_extra/NET2/footer-toast-published-image-1.png
爬取失敗
None


# 網站爬圖 by Icrawler

In [121]:
from icrawler.builtin import BaiduImageCrawler
from icrawler.builtin import BingImageCrawler
from icrawler.builtin import GoogleImageCrawler

"""
parser_threads : 解析器线程数目，最大为cpu数目
downloader_threads : 下载线程数目，最大为cpu数目
storage : 存储地址，使用字典格式。key为root_dir
keyword : 浏览器搜索框输入的关键词
max_num : 最大下载图片数目
"""

import time
import os

KEY_WORD   = '和泉紗霧'
KEY_LABEL  = '0'
IMAGE_NUM  = 50
BROWSER_ID = [1, 0, 0]  # 0:Google  1:Bing  2:Baidu

LABELS = [
    '0_Izumi Sagiri_和泉紗霧',
    '1_Izumi Masamune_和泉正宗',
    '2_Yamada Erufu_山田妖精',
    '3_Jinno Megumi_神野惠',
    '4_Muramasa Senju_千壽村征',
    '5_Takasago Tomoe_高砂智惠',
    '6_Kagurazaka Ayame_神樂坂菖蒲',
]

TIME = time.strftime ("_%Y-%m-%d_%H'%M'%S", time.localtime())


def Crawler_Recognition (IP, OP, L):
    
    dIMAGES = os.listdir(IP)
    
    if not os.path.exists(OP):
        os.mkdir(OP)
    
    imageT_i = 0
    imageF_i = 0
    for dIMAGE in dIMAGES:
        
        fname, ftype = os.path.splitext(dIMAGE)
        ftype = '.jpg'
        
        img, faces = Get_Image_Animeface_Detection (IP + '/' + dIMAGE)
        img_show   = cv2.imread(IP + '/' + dIMAGE)
        
        img_T = 0
        for (x, y, w, h) in faces:
            
            cv2.rectangle(img_show,(x,y),(x+w,y+h),(255,0,0),5)
            
            ans_str = Get_Image_Animeface_Recognition (cv2.resize(img[y:y+h, x:x+w], (INPUT_SIZE, INPUT_SIZE)), CROP_LIST, PLAN)
            print('ans : ', ans_str)
            
            if (ans_str == L):
                img_T = 1
                cv2.rectangle(img_show,(x,y),(x+w,y+h),(0,0,255),5)
                break
        
        os.remove  (IP + '/' + dIMAGE)
        if(img_T == 1):
            print(OP + '/' + str(imageT_i) + ftype)
            cv2.imwrite(OP + '/' + str(imageT_i) + ftype, img_show)
            imageT_i += 1
        elif img_show is not None:
            print(IP + '/' + str(imageF_i) + ftype)
            cv2.imwrite(IP + '/' + str(imageF_i) + ftype, img_show)
            imageF_i += 1
            
        print("-----------------------------------------------------")

# Google
if (BROWSER_ID[0] == 1):
    PATH = 'data_extra/Google'
    google_storage = {'root_dir':'/Users/user/Desktop/jupyter/PatternProject/' + PATH + '/dealing'}
    google_crawler = GoogleImageCrawler(parser_threads=2, downloader_threads=4, storage=google_storage)
    google_crawler.crawl(keyword=KEY_WORD, max_num=IMAGE_NUM)
    
    Crawler_Recognition (PATH + '/dealing', PATH + '/dealing/dealing', KEY_LABEL)
    os.rename(PATH + '/dealing/dealing', PATH + '/dealing/' + LABELS[int(KEY_LABEL)])
    os.rename(PATH + '/dealing'        , PATH + '/' + KEY_WORD + TIME)

# Bing
if (BROWSER_ID[1] == 1):
    PATH = 'data_extra/Bing'
    bing_storage = {'root_dir':'/Users/user/Desktop/jupyter/PatternProject/' + PATH + '/dealing'}
    bing_crawler = BingImageCrawler(parser_threads=2,  downloader_threads=4, storage=bing_storage)
    bing_crawler.crawl(keyword=KEY_WORD, max_num=IMAGE_NUM)
    
    Crawler_Recognition (PATH + '/dealing', PATH + '/dealing/dealing', KEY_LABEL)
    os.rename(PATH + '/dealing/dealing', PATH + '/dealing/' + LABELS[int(KEY_LABEL)])
    os.rename(PATH + '/dealing'        , PATH + '/' + KEY_WORD + TIME)

# Baidu
if (BROWSER_ID[2] == 1):
    PATH = 'data_extra/Baidu'
    baidu_storage = {'root_dir':'/Users/user/Desktop/jupyter/PatternProject/' + PATH + '/dealing'}
    baidu_crawler = BaiduImageCrawler(parser_threads=2, downloader_threads=4, storage=baidu_storage)
    baidu_crawler.crawl(keyword=KEY_WORD, max_num=IMAGE_NUM)
    
    Crawler_Recognition (PATH + '/dealing', PATH + '/dealing/dealing', KEY_LABEL)
    os.rename(PATH + '/dealing/dealing', PATH + '/dealing/' + LABELS[int(KEY_LABEL)])
    os.rename(PATH + '/dealing'        , PATH + '/' + KEY_WORD + TIME)


2018-12-29 07:01:22,279 - INFO - icrawler.crawler - start crawling...
2018-12-29 07:01:22,280 - INFO - icrawler.crawler - starting 1 feeder threads...
2018-12-29 07:01:22,285 - INFO - feeder - thread feeder-001 exit
2018-12-29 07:01:22,294 - INFO - icrawler.crawler - starting 2 parser threads...
2018-12-29 07:01:22,299 - INFO - icrawler.crawler - starting 4 downloader threads...
2018-12-29 07:01:22,574 - INFO - parser - parsing result page https://www.google.com/search?ijn=0&tbs=&start=0&tbm=isch&q=%E5%92%8C%E6%B3%89%E7%B4%97%E9%9C%A7
2018-12-29 07:01:22,758 - INFO - downloader - image #1	https://i.ytimg.com/vi/WBq_pOFu2Ew/maxresdefault.jpg
2018-12-29 07:01:22,767 - INFO - downloader - image #2	https://i2.kknews.cc/SIG=1ttlkeu/213600046424q8rs774n.jpg
2018-12-29 07:01:22,814 - INFO - downloader - image #3	https://i2.kknews.cc/SIG=34lsr03/2n450004so1r0spq2p49.jpg
2018-12-29 07:01:22,820 - INFO - downloader - image #4	https://im1.book.com.tw/image/getImage?i=https://www.books.com.tw/img/

2018-12-29 07:01:30,149 - INFO - downloader - thread downloader-004 exit
2018-12-29 07:01:30,819 - ERROR - downloader - Exception caught when downloading file https://wall.lingcell.com/funclib/preview_hd/oid/748, error: HTTPSConnectionPool(host='wall.lingcell.com', port=443): Max retries exceeded with url: /funclib/preview_hd/oid/748 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x0000018707E4C780>: Failed to establish a new connection: [WinError 10061] 無法連線，因為目標電腦拒絕連線。',)), remaining retry times: 0
2018-12-29 07:01:30,823 - INFO - downloader - downloaded images reach max num, thread downloader-002 is ready to exit
2018-12-29 07:01:30,824 - INFO - downloader - thread downloader-002 exit
2018-12-29 07:01:31,760 - INFO - parser - downloaded image reached max num, thread parser-001 is ready to exit
2018-12-29 07:01:31,761 - INFO - parser - thread parser-001 exit
2018-12-29 07:01:35,034 - INFO - downloader - downloaded images reach max num, thread dow

(720, 1280, 3)
NO  :  [1.0000000e+00 6.6733868e-17 2.4926654e-12 8.7307768e-16 2.2749756e-20
 4.7502031e-19 1.5144330e-11]
MID  :  [1.0000000e+00 5.7688894e-15 5.6326426e-11 3.9255967e-17 1.1090185e-19
 4.1219901e-15 1.9319601e-11]
EDGE  :  [9.98791039e-01 2.37787346e-04 2.47461521e-05 1.11761062e-06
 5.87064249e-04 2.37480024e-04 1.20731595e-04]
VOTE :  [3. 0. 0. 0. 0. 0. 0.]
ans :  0
data_extra/Google/dealing/dealing/0.jpg
-----------------------------------------------------
(895, 640, 3)
NO  :  [1.0000000e+00 6.4676275e-16 2.2857227e-09 4.8575261e-13 1.8416297e-19
 2.0429911e-18 4.3408149e-10]
MID  :  [9.99999404e-01 1.08573925e-12 5.60143121e-07 1.05339436e-13
 3.13583074e-15 1.19165432e-14 2.46850149e-08]
EDGE  :  [2.3281494e-01 6.1657599e-09 2.8062174e-01 1.9350317e-05 2.5892010e-12
 4.6582809e-09 4.8654398e-01]
VOTE :  [2. 0. 0. 0. 0. 0. 0.]
ans :  0
data_extra/Google/dealing/dealing/1.jpg
-----------------------------------------------------
(687, 546, 3)
NO  :  [1.0000000e+00

NO  :  [9.9998021e-01 1.4788765e-18 1.9804904e-05 5.0040007e-20 1.1871897e-21
 1.6532339e-17 1.4370805e-10]
MID  :  [1.0000000e+00 1.4315952e-19 7.4962866e-12 3.0068924e-21 3.8679983e-23
 1.1154365e-14 3.3747529e-11]
EDGE  :  [5.9584464e-04 2.4290926e-13 9.9923432e-01 6.1080985e-10 1.2137443e-17
 1.6983644e-04 1.2120369e-10]
VOTE :  [2. 0. 1. 0. 0. 0. 0.]
ans :  0
data_extra/Google/dealing/dealing/18.jpg
-----------------------------------------------------
(800, 480, 3)
NO  :  [1.0000000e+00 2.1807146e-13 1.3367802e-08 1.9353154e-12 1.2053058e-15
 4.1491382e-16 3.3459846e-09]
MID  :  [1.0000000e+00 7.1582063e-12 5.7925242e-09 3.7726383e-14 1.3571334e-14
 1.8121084e-13 5.5435878e-09]
EDGE  :  [4.0299731e-01 4.2918572e-08 5.8342028e-01 7.3677934e-06 8.6345290e-18
 1.6459753e-10 1.3575008e-02]
VOTE :  [2. 0. 1. 0. 0. 0. 0.]
ans :  0
data_extra/Google/dealing/dealing/19.jpg
-----------------------------------------------------
(400, 400, 3)
NO  :  [1.0000000e+00 7.4450418e-17 5.5872498e-1

NO  :  [1.0000000e+00 5.5365726e-16 2.3032961e-11 2.9724922e-16 8.8885358e-20
 4.9991057e-18 7.3598849e-11]
MID  :  [1.0000000e+00 3.2094743e-15 1.7002856e-11 1.4197137e-17 1.4145927e-19
 7.4378851e-14 5.8006887e-11]
EDGE  :  [1.7535577e-03 1.5952827e-12 9.9768603e-01 3.2510977e-10 8.2603387e-20
 5.7995095e-11 5.6044612e-04]
VOTE :  [2. 0. 1. 0. 0. 0. 0.]
ans :  0
data_extra/Google/dealing/dealing/34.jpg
-----------------------------------------------------
(1020, 835, 3)
NO  :  [9.9998939e-01 2.6353837e-16 1.0414136e-05 5.2198350e-13 4.6748006e-17
 1.6619625e-17 1.9738204e-07]
MID  :  [1.0000000e+00 4.4492703e-16 1.0371058e-08 3.9871711e-14 4.5646092e-16
 1.8561634e-14 6.9669168e-09]
EDGE  :  [1.5721194e-12 4.3215577e-14 1.0000000e+00 1.7099105e-09 4.7367597e-26
 3.2278985e-11 2.9854590e-08]
VOTE :  [2. 0. 1. 0. 0. 0. 0.]
ans :  0
data_extra/Google/dealing/dealing/35.jpg
-----------------------------------------------------
(650, 700, 3)
NO  :  [1.0000000e+00 1.0416652e-16 1.3556241e-

# Free Coding

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
import time
 
# 格式化成2016-03-20 11:45:39形式
print (time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))

In [15]:
import numpy as np

PPP = 10

def AA ():
    global PPP
    
    PPP = 111

AA()
print('ans', PPP)

ans 111


In [23]:
import numpy as np
from keras.models import load_model
NN = []
NN.append(load_model('NO' + '_' + '2DCNN' + '.h5'))
NN.append([1,2,3,4,5])
print(NN[0])

ValueError: Unknown layer:name

In [64]:
import cv2
print(type(np.array([])))
if (type(np.array([])) == <class 'numpy.ndarray'>):
    print("A")

SyntaxError: invalid syntax (<ipython-input-64-f95c0d486a31>, line 3)